***Data: https://www.kaggle.com/datasets/thedevastator/airlines-traffic-passenger-statistics***

In [101]:
import pandas as pd
import plotly.express as px
import datetime

df = pd.read_csv('air_traffic.csv', index_col="index")
df_airlines = df.drop(columns=["Published Airline", "Published Airline IATA Code", "GEO Summary", "Price Category Code", "Terminal", "Boarding Area", "Adjusted Activity Type Code", "Adjusted Passenger Count", "Activity Type Code"])
df_airlines

,Activity Period,Operating Airline,Operating Airline IATA Code,GEO Region,Passenger Count,Year,Month
index,,,,,,,
0,200507,ATA Airlines,TZ,US,27271,2005,July
1,200507,ATA Airlines,TZ,US,29131,2005,July
2,200507,ATA Airlines,TZ,US,5415,2005,July
3,200507,Air Canada,AC,Canada,35156,2005,July
4,200507,Air Canada,AC,Canada,34090,2005,July
...,...,...,...,...,...,...,...
15002,201603,Virgin America,VX,US,194636,2016,March
15003,201603,Virgin America,VX,Mexico,4189,2016,March
15004,201603,Virgin America,VX,Mexico,4693,2016,March


In [102]:
# Let's select a few airlines to plot its data
selected_airlines = ["China Airlines", "Air France", "Virgin Atlantic", "British Airways", "Turkish Airlines"]

df_airlines_selected = df_airlines[df_airlines["Operating Airline"].isin(selected_airlines)]
df_airlines_selected

,Activity Period,Operating Airline,Operating Airline IATA Code,GEO Region,Passenger Count,Year,Month
index,,,,,,,
7,200507,Air France,AF,Europe,12050,2005,July
8,200507,Air France,AF,Europe,11638,2005,July
33,200507,British Airways,BA,Europe,20632,2005,July
34,200507,British Airways,BA,Europe,21686,2005,July
37,200507,China Airlines,CI,Asia,13638,2005,July
...,...,...,...,...,...,...,...
14915,201603,China Airlines,CI,Asia,9853,2016,March
14974,201603,Turkish Airlines,TK,Europe,8865,2016,March
14975,201603,Turkish Airlines,TK,Europe,8729,2016,March


In [103]:
df_airlines_grouped = df_airlines_selected.groupby(["Activity Period", "Operating Airline IATA Code", "Operating Airline", "Year", "Month"], as_index=False)
df_airlines_prepared = df_airlines_grouped["Passenger Count"].sum()
df_airlines_prepared

,Activity Period,Operating Airline IATA Code,Operating Airline,Year,Month,Passenger Count
0,200507,AF,Air France,2005,July,23688
1,200507,BA,British Airways,2005,July,42318
2,200507,CI,China Airlines,2005,July,24893
3,200507,VS,Virgin Atlantic,2005,July,22455
4,200508,AF,Air France,2005,August,22961
...,...,...,...,...,...,...
523,201603,AF,Air France,2016,March,20546
524,201603,BA,British Airways,2016,March,37826
525,201603,CI,China Airlines,2016,March,19387
526,201603,TK,Turkish Airlines,2016,March,17594


In [104]:
df_converted = df_airlines_prepared.pivot(index='Activity Period', columns='Operating Airline', values='Passenger Count').rename_axis(None, axis=1).reset_index()
df_converted['Activity Period'] = df_converted["Activity Period"].apply(lambda x: pd.to_datetime(str(x), format='%Y%m').strftime("%Y-%m"))
df_converted

,Activity Period,Air France,British Airways,China Airlines,Turkish Airlines,Virgin Atlantic
0,2005-07,23688.0,42318.0,24893.0,NaN,22455.0
1,2005-08,22961.0,40396.0,25582.0,NaN,21757.0
2,2005-09,22068.0,41090.0,18640.0,NaN,22707.0
3,2005-10,21311.0,38441.0,19173.0,NaN,19355.0
4,2005-11,14695.0,30818.0,19188.0,NaN,16195.0
...,...,...,...,...,...,...
124,2015-11,17729.0,38699.0,16353.0,17549.0,19159.0
125,2015-12,17930.0,36013.0,18021.0,16870.0,21571.0
126,2016-01,17853.0,38399.0,19993.0,17199.0,20948.0
127,2016-02,15876.0,33703.0,17481.0,15800.0,16035.0


In [105]:
fig = px.line(df_converted, x="Activity Period", y=df_converted.columns,
              hover_data={"Activity Period": "|%B, %Y"},
              title='Airline traffic in US')
fig.update_xaxes(
    dtick="M4",
    tickformat="%B %Y")
fig.show()